In [43]:
import os
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import itertools
import numpy as np
import json

import map_processing as mp 

from groq import Groq 
from dotenv import load_dotenv



In [44]:
# configurations

vehicles = [
    {"id": 1, "type": "fire", "name": "firefighter_1", "station_address": "2241 Jerrold Ave, San Francisco, CA 94124, USA", "station_latitude": 37.755836, "station_longitude": -122.428429, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 2, "type": "fire", "name": "firefighter_2", "station_address": "2150 California St, San Francisco, CA 94115", "station_latitude": 37.784279, "station_longitude": -122.407234, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 3, "type": "fire", "name": "firefighter_3", "station_address": "441 12th Ave, San Francisco, CA 94118", "station_latitude": 37.779847, "station_longitude": -122.471177, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 4, "type": "police", "name": "police_1", "station_address": "766 Vallejo St, San Francisco, CA 94133", "station_latitude": 37.798729, "station_longitude": -122.409912, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 5, "type": "police", "name": "police_2", "station_address": "630 Valencia St, San Francisco, CA 94110", "station_latitude": 37.762852, "station_longitude": -122.421944, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 6, "type": "police", "name": "police_3", "station_address": "2345 24th Ave, San Francisco, CA 94116", "station_latitude": 37.743732, "station_longitude": -122.481461, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 7, "type": "med", "name": "medical_1", "station_address": "698 Second St, San Francisco, CA 94107", "station_latitude": 37.780682, "station_longitude": -122.390854, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 8, "type": "med", "name": "medical_2", "station_address": "1300 Illinois St, San Francisco, CA 94107", "station_latitude": 37.754686, "station_longitude": -122.387411, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 9, "type": "med", "name": "medical_3", "station_address": "2570 Bush St, San Francisco, CA 94115", "station_latitude": 37.786523, "station_longitude": -122.439595, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 10, "type": "fire", "name": "firefighter_12", "station_address": "2241 Jerrold Ave, San Francisco, CA 94124, USA", "station_latitude": 37.755836, "station_longitude": -122.428429, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 11, "type": "fire", "name": "firefighter_13", "station_address": "2241 Jerrold Ave, San Francisco, CA 94124, USA", "station_latitude": 37.755836, "station_longitude": -122.428429, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 12, "type": "fire", "name": "firefighter_22", "station_address": "2150 California St, San Francisco, CA 94115", "station_latitude": 37.784279, "station_longitude": -122.407234, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 15, "type": "police", "name": "police_12", "station_address": "766 Vallejo St, San Francisco, CA 94133", "station_latitude": 37.798729, "station_longitude": -122.409912, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 18, "type": "police", "name": "police_32", "station_address": "2345 24th Ave, San Francisco, CA 94116", "station_latitude": 37.743732, "station_longitude": -122.481461, "status": "ready", "target_lat": None, "target_lng": None},
    {"id": 21, "type": "med", "name": "medical_22", "station_address": "1300 Illinois St, San Francisco, CA 94107", "station_latitude": 37.754686, "station_longitude": -122.387411, "status": "ready", "target_lat": None, "target_lng": None}
]

# Define Fixed Origin Location (e.g., incident location)
hotel = [37.7821, -122.4047] # Latitude, Longitude

In [45]:
load_dotenv()

# Get the API key from environment variables
API_KEY = os.getenv("GROQ_API_KEY")

# Initialize Groq client
client = None
if not API_KEY:
    print("API_KEY not found. Please set the GROQ_API_KEY environment variable or place it in a .env file.")
else:
    try:
        client = Groq(api_key=API_KEY) # Initialize Groq client with the key
        print("Groq client initialized successfully.")
        # Define the model to use (one that supports tool use)
        GROQ_MODEL = 'llama-3.3-70b-versatile'
        print(f"Using Groq model: {GROQ_MODEL}")
    except Exception as e:
        print(f"Error initializing Groq client: {e}")
        client = None # Ensure client is None if initialization fails

print("\nSetup Complete.")


Groq client initialized successfully.
Using Groq model: llama-3.3-70b-versatile

Setup Complete.


In [ ]:
# Load and Prepare Graph

place_name = "San Francisco, California, USA"
network_type = "drive"

print(f"Loading unprojected graph for {place_name}...")
try:
    G_undirected = ox.graph_from_place(place_name, network_type=network_type)
    print("Unprojected graph loaded.")

    # Create the projected graph (for calculations)
    print("Projecting graph for calculations...")
    G_proj = ox.project_graph(G_undirected)
    print("Graph projected.")

    print("\nGraphs prepared:")
    print(f"  G_undirected (for plotting): {len(G_undirected.nodes)} nodes, {len(G_undirected.edges)} edges")
    print(f"  G_proj (for calculations): {len(G_proj.nodes)} nodes, {len(G_proj.edges)} edges")

except Exception as e:
    print(f"Error loading or projecting graph: {e}")
    G_undirected = None
    G_proj = None


Loading unprojected graph for San Francisco, California, USA...


/opt/anaconda3/envs/cerebralVal/lib/python3.13/site-packages/shapely/constructive.py:180: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/anaconda3/envs/cerebralVal/lib/python3.13/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/cerebralVal/lib/python3.13/site-packages/shapely/set_operations.py:334: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/anaconda3/envs/cerebralVal/lib/python3.13/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/cerebralVal/lib/python3.13/site-packages/shapely/set_operations.py:334: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


Unprojected graph loaded.
Projecting graph for calculations...
Graph projected.

Graphs prepared:
  G_undirected (for plotting): 10006 nodes, 27580 edges
  G_proj (for calculations): 10006 nodes, 27580 edges


In [47]:
# Pre-calculate Paths (Context for Tool)
# Calculate paths from the **fixed incident location (hotel)** to all vehicle stations.
# This data will be used by our Groq tool later to determine closeness relative to a *new* target point.


results_list = [] # Initialize results list

if G_proj and 'hotel' in locals():
    print(f"Calculating initial paths from Incident Location {hotel} to all stations...")
    print("-" * 40)

    # Find the nearest node to the fixed incident location ONCE
    try:
        # Use the imported function
        incident_node = mp.find_nearest_node(G_proj, hotel[0], hotel[1])
        print(f"Fixed Incident Node (near hotel {hotel}): {incident_node}\n")

        # Iterate through each vehicle
        for index, vehicle in enumerate(vehicles):
            vehicle_id = vehicle.get('id', 'N/A')
            vehicle_name = vehicle.get('name', 'N/A')
            print(f"Processing Vehicle {index + 1}/{len(vehicles)}: ID={vehicle_id}, Name='{vehicle_name}'")

            station_lat = vehicle.get('station_latitude')
            station_lon = vehicle.get('station_longitude')

            # Initialize result fields
            dispatch_station_coords = None
            station_node = None
            shortest_path_nodes = None
            path_node_count = None
            path_length_meters = None
            error_message = None

            if station_lat is None or station_lon is None:
                error_message = "Missing coordinates"
                print(f"  Skipping Vehicle ID {vehicle_id}: {error_message}.")
            else:
                dispatch_station_coords = [station_lat, station_lon]
                print(f"  Station Coords: {dispatch_station_coords}")
                try:
                    # Use the imported function
                    station_node = mp.find_nearest_node(G_proj, station_lat, station_lon)
                    print(f"  Station Node: {station_node}")

                    if incident_node == station_node:
                        shortest_path_nodes = [incident_node]
                        path_node_count = 1
                        path_length_meters = 0.0
                    else:
                        # Use the imported function
                        shortest_path_nodes = mp.find_shortest_path(G_proj, incident_node, station_node, weight='length')
                        if shortest_path_nodes:
                            path_node_count = len(shortest_path_nodes)
                            # Use the imported function (assuming it exists in map_processing.py)
                            # If calculate_path_length is not in map_processing.py, define it or adapt
                            if hasattr(mp, 'calculate_path_length'):
                                path_length_meters = mp.calculate_path_length(G_proj, shortest_path_nodes, weight='length')
                            else: # Fallback if function missing from module
                                print("  Warning: mp.calculate_path_length not found. Calculating length manually.")
                                try:
                                     edge_lengths = ox.utils_graph.get_route_edge_attributes(G_proj, shortest_path_nodes, 'length')
                                     path_length_meters = sum(edge_lengths)
                                except Exception: path_length_meters = None

                            print(f"  Initial path found: {path_node_count} nodes, Length: {path_length_meters:.2f}m" if path_length_meters is not None else f"  Initial path found: {path_node_count} nodes, Length: N/A")
                        else:
                             path_node_count = 0
                             error_message = "No path found"
                except Exception as e:
                    print(f"  ERROR processing path for vehicle ID {vehicle_id}: {e}")
                    error_message = str(e)
                    if station_node is None: station_node = "Error finding node"

            # Store results including the station node (important for the tool)
            results_list.append({
                "vehicle_id": vehicle_id, "vehicle_name": vehicle_name,
                "station_coords": dispatch_station_coords, "station_node": station_node, # Store the station node ID
                "initial_path_nodes": shortest_path_nodes, # Path from incident
                "initial_path_length_meters": path_length_meters, # Length from incident
                "error": error_message
            })
            print("-" * 40)
        print("\nFinished calculating initial paths.")

    except Exception as e:
        print(f"FATAL ERROR: Could not find nearest node for incident location {hotel}. Cannot pre-calculate paths. Error: {e}")

else:
    print("Skipping initial path calculation: G_proj graph not loaded or hotel location not defined.")

# Convert results to DataFrame for easier access later (optional)
if results_list:
    results_df = pd.DataFrame(results_list)
    print("\nInitial path results stored in 'results_list' and 'results_df'.")
    # print(results_df.head()) # Optionally display head of dataframe
else:
    results_df = pd.DataFrame() # Create empty dataframe if no results

Calculating initial paths from Incident Location [37.7821, -122.4047] to all stations...
----------------------------------------
Fixed Incident Node (near hotel [37.7821, -122.4047]): 6873099368

Processing Vehicle 1/15: ID=1, Name='firefighter_1'
  Station Coords: [37.755836, -122.428429]
  Station Node: 65318058
  ERROR processing path for vehicle ID 1: find_shortest_path() got an unexpected keyword argument 'weight'
----------------------------------------
Processing Vehicle 2/15: ID=2, Name='firefighter_2'
  Station Coords: [37.784279, -122.407234]
  Station Node: 7293111110
  ERROR processing path for vehicle ID 2: find_shortest_path() got an unexpected keyword argument 'weight'
----------------------------------------
Processing Vehicle 3/15: ID=3, Name='firefighter_3'
  Station Coords: [37.779847, -122.471177]
  Station Node: 258911613
  ERROR processing path for vehicle ID 3: find_shortest_path() got an unexpected keyword argument 'weight'
-------------------------------------

In [48]:
# Define Groq Tool: `find_closest_vehicle_station`
# This tool allows Groq to find the vehicle station closest (by shortest path distance) to a *new* specified latitude and longitude. It uses the pre-calculated station node information and **imported helper functions**.


# --- Tool's Python Function ---
def find_closest_vehicle_station(target_lat: float, target_lon: float):
    """
    Finds the vehicle station closest to the given target latitude and longitude
    by calculating shortest path distances from the target to each station's nearest node.
    Uses helper functions imported from map_processing.py.

    Args:
        target_lat: Latitude of the target location.
        target_lon: Longitude of the target location.

    Returns:
        A JSON string containing the details of the closest vehicle station
        (ID, name, station coordinates, distance in meters) or an error message.
    """
    print(f"\n--- Tool Execution: find_closest_vehicle_station ---")
    print(f"Target Location: Lat={target_lat}, Lon={target_lon}")

    # Ensure necessary data is available
    if G_proj is None:
        return json.dumps({"error": "Projected graph (G_proj) is not available."})
    if 'results_df' not in globals() or results_df.empty:
         return json.dumps({"error": "Pre-calculated vehicle station data (results_df) is empty or not available."})

    closest_vehicle_info = None
    min_distance = float('inf')

    try:
        # Find the nearest node in the projected graph to the target location
        # Use the imported function
        target_node = mp.find_nearest_node(G_proj, target_lat, target_lon)
        print(f"Nearest node to target: {target_node}")

        # Iterate through the pre-calculated results for each vehicle
        for index, vehicle_data in results_df.iterrows():
            station_node = vehicle_data.get('station_node')
            vehicle_id = vehicle_data.get('vehicle_id')
            vehicle_name = vehicle_data.get('vehicle_name')
            station_coords = vehicle_data.get('station_coords')

            # Skip if station node wasn't found previously or is invalid
            if station_node is None or not isinstance(station_node, (int, np.int64)):
                print(f"  Skipping Vehicle ID {vehicle_id}: Invalid or missing station node ({station_node}).")
                continue

            station_node = int(station_node)

            print(f"  Calculating path from target node {target_node} to station node {station_node} (Vehicle ID: {vehicle_id})...")

            # Calculate shortest path from the target node to this station node
            # Use the imported function
            path_nodes = mp.find_shortest_path(G_proj, target_node, station_node, weight='length')

            if path_nodes:
                # Calculate the length of this path
                # Use the imported function (check if exists)
                if hasattr(mp, 'calculate_path_length'):
                    distance_meters = mp.calculate_path_length(G_proj, path_nodes, weight='length')
                else: # Fallback
                    print("  Warning: mp.calculate_path_length not found. Calculating length manually.")
                    try:
                         edge_lengths = ox.utils_graph.get_route_edge_attributes(G_proj, path_nodes, 'length')
                         distance_meters = sum(edge_lengths)
                    except Exception: distance_meters = None

                print(f"    Path found. Distance: {distance_meters:.2f}m" if distance_meters is not None else "    Path found. Distance: N/A")

                if distance_meters is not None and distance_meters < min_distance:
                    min_distance = distance_meters
                    closest_vehicle_info = {
                        "vehicle_id": vehicle_id,
                        "vehicle_name": vehicle_name,
                        "station_coords": station_coords,
                        "station_node": station_node,
                        "distance_meters": round(distance_meters, 2),
                        "path_node_count": len(path_nodes)
                    }
            else:
                print(f"    No path found from target node {target_node} to station node {station_node}.")

        # --- Prepare and return result ---
        if closest_vehicle_info:
            print(f"Closest vehicle found: ID={closest_vehicle_info['vehicle_id']}, Distance={closest_vehicle_info['distance_meters']}m")
            return json.dumps(closest_vehicle_info)
        else:
            print("Could not determine the closest vehicle station.")
            return json.dumps({"error": "Could not determine the closest vehicle station. No valid paths found."})

    except Exception as e:
        print(f"Error during tool execution: {e}")
        return json.dumps({"error": f"An error occurred during tool execution: {str(e)}"})
    finally:
        print(f"--- Tool Execution Finished ---")


# --- Tool Schema for Groq ---
# (Schema remains the same, only the underlying function implementation changes)
tools_definition = [
    {
        "type": "function",
        "function": {
            "name": "find_closest_vehicle_station",
            "description": "Determines which vehicle station is closest (by shortest driving distance) to a given latitude and longitude coordinate. Uses pre-calculated graph data.",
            "parameters": {
                "type": "object",
                "properties": {
                    "target_lat": {
                        "type": "number",
                        "description": "The latitude of the target location.",
                    },
                    "target_lon": {
                        "type": "number",
                        "description": "The longitude of the target location.",
                    },
                },
                "required": ["target_lat", "target_lon"],
            },
        },
    }
]

# Map tool name to the actual Python function (defined in this cell)
available_functions = {
    "find_closest_vehicle_station": find_closest_vehicle_station,
}

print("Groq tool 'find_closest_vehicle_station' (using imported helpers) and its schema defined.")


Groq tool 'find_closest_vehicle_station' (using imported helpers) and its schema defined.


In [49]:
# 6. Groq Interaction Function

# This function remains the same as it calls the tool function defined above,
# which now internally uses the imported helpers.

def ask_groq_about_routing(user_query: str):
    """
    Sends a query to Groq, handles potential tool calls for routing,
    and returns the final natural language response.
    """
    if client is None:
        return "Groq client is not initialized. Cannot process query."
    if G_proj is None:
        return "Projected graph (G_proj) is not available. Cannot process query."
    if 'results_df' not in globals() or results_df.empty:
         return "Pre-calculated vehicle station data is not available. Cannot process query."


    print(f"\n>>> User Query: {user_query}")

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant knowledgeable about emergency vehicle locations and routing in San Francisco. Use the available tools to answer questions about vehicle proximity to specific locations based on driving distance."
        },
        {
            "role": "user",
            "content": user_query,
        }
    ]

    try:
        # --- First API Call: Let Groq decide if a tool is needed ---
        print("\n--- Sending query to Groq (potential tool use)... ---")
        response = client.chat.completions.create(
            model=GROQ_MODEL,
            messages=messages,
            tools=tools_definition,
            tool_choice="auto", # Let Groq decide
            max_tokens=4096
        )

        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        # --- Check if Groq decided to use a tool ---
        if tool_calls:
            print(f"\n--- Groq requested tool call(s): ---")
            messages.append(response_message) # Add assistant's reply (with tool call request) to history

            # --- Execute Tool(s) ---
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                print(f"  Tool: {function_name}")
                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    try:
                        function_args = json.loads(tool_call.function.arguments)
                        print(f"  Arguments: {function_args}")

                        # Call the actual Python function
                        function_response = function_to_call(**function_args) # Use dictionary unpacking

                        # Add tool execution result to message history
                        messages.append(
                            {
                                "tool_call_id": tool_call.id,
                                "role": "tool",
                                "name": function_name,
                                "content": function_response, # The JSON string returned by the tool
                            }
                        )
                        print(f"  Tool Response: {function_response}")
                    except Exception as e:
                        print(f"  Error executing tool {function_name}: {e}")
                        # Optionally add an error message back to Groq
                        messages.append({
                            "tool_call_id": tool_call.id, "role": "tool", "name": function_name,
                            "content": json.dumps({"error": f"Failed to execute tool: {str(e)}"})
                        })
                else:
                    print(f"  Error: Unknown tool '{function_name}' requested.")
                    # Add error message back
                    messages.append({
                        "tool_call_id": tool_call.id, "role": "tool", "name": function_name,
                        "content": json.dumps({"error": f"Unknown tool '{function_name}' requested."})
                    })

            # --- Second API Call: Get final response based on tool results ---
            print("\n--- Sending updated conversation to Groq for final response... ---")
            second_response = client.chat.completions.create(
                model=GROQ_MODEL,
                messages=messages,
                max_tokens=4096
                # No tools needed here, just summarizing
            )
            final_response = second_response.choices[0].message.content
            print("\n<<< Groq Final Response (after tool use):")
            print(final_response)
            return final_response

        else:
            # --- No tool call requested ---
            final_response = response_message.content
            print("\n<<< Groq Response (no tool used):")
            print(final_response)
            return final_response

    except Exception as e:
        print(f"\nAn error occurred during Groq API interaction: {e}")
        return f"Error interacting with Groq API: {e}"

print("Groq interaction function 'ask_groq_about_routing' defined.")

Groq interaction function 'ask_groq_about_routing' defined.


In [50]:
# 7. Example Query using Groq Tool

# Ensure client is initialized and required data is available before querying
# Check if results_df exists and is not empty
results_available = 'results_df' in globals() and not results_df.empty

if client and G_proj and results_available:
    # Example query asking about the location defined by the 'hotel' variable
    query = f"Which vehicle station is closest by driving distance to the incident at latitude {hotel[0]}, longitude {hotel[1]}?"
    groq_response = ask_groq_about_routing(query)

    print("\n--- Example Query Result ---")
    print(f"Query: {query}")
    print(f"Groq's Answer:\n{groq_response}")

    # Example query asking about a different location
    query_2 = "Find the closest vehicle station to latitude 37.76, longitude -122.45"
    groq_response_2 = ask_groq_about_routing(query_2)
    print("\n--- Example Query 2 Result ---")
    print(f"Query: {query_2}")
    print(f"Groq's Answer:\n{groq_response_2}")

else:
    print("\nSkipping example query: Groq client not initialized, graph not loaded, or initial results not calculated.")



>>> User Query: Which vehicle station is closest by driving distance to the incident at latitude 37.7821, longitude -122.4047?

--- Sending query to Groq (potential tool use)... ---

--- Groq requested tool call(s): ---
  Tool: find_closest_vehicle_station
  Arguments: {'target_lat': 37.7821, 'target_lon': -122.4047}

--- Tool Execution: find_closest_vehicle_station ---
Target Location: Lat=37.7821, Lon=-122.4047
Nearest node to target: 6873099368
  Calculating path from target node 6873099368 to station node 65318058 (Vehicle ID: 1)...
Error during tool execution: find_shortest_path() got an unexpected keyword argument 'weight'
--- Tool Execution Finished ---
  Tool Response: {"error": "An error occurred during tool execution: find_shortest_path() got an unexpected keyword argument 'weight'"}

--- Sending updated conversation to Groq for final response... ---

<<< Groq Final Response (after tool use):
Unfortunately, I was unable to find the closest vehicle station to the incident at 

In [51]:
# 8. Plotting (Optional)
# Visualize the initial routes calculated from the incident location.

print("\nPreparing plot...")

# Check if results exist and are not empty
if 'results_list' not in locals() or not results_list:
    print("Cannot plot: 'results_list' is not defined or empty.")
elif 'G_undirected' not in locals() or G_undirected is None:
    print("Cannot plot: 'G_undirected' graph is not defined.")
else:
    # Define a list of colors for the paths
    color_list = ['cyan', 'lime', 'magenta', 'yellow', 'orange', 'red', 'blue',
                  'springgreen', 'fuchsia', 'gold', 'darkorange', 'deeppink', 'dodgerblue', 'greenyellow']
    color_cycle = itertools.cycle(color_list)

    routes_to_plot = []
    route_colors = []
    station_markers = [] # To store coords, colors, and labels

    print("Filtering valid initial paths for plotting...")
    for result in results_list:
        # Use the 'initial_path_nodes' calculated in step 4
        path = result.get('initial_path_nodes')
        vehicle_id = result.get('vehicle_id', 'N/A')

        if path and isinstance(path, (list, tuple)) and len(path) > 1:
            routes_to_plot.append(path)
            color = next(color_cycle)
            route_colors.append(color)
            station_markers.append({
                "coords": result.get("station_coords"),
                "color": color,
                "label": f"V{vehicle_id}"
            })
            # print(f"  Added initial path for Vehicle ID {vehicle_id} to plot.") # Less verbose
        # (Skipping print statements for non-plotted paths for brevity)

    if not routes_to_plot:
        print("\nNo valid initial paths found to plot.")
    else:
        print(f"\nPlotting {len(routes_to_plot)} initial paths from incident location...")
        try:
            # Ensure G_undirected has lat/lon for plotting
            fig, ax = ox.plot_graph_routes(
                G_undirected,
                routes_to_plot,
                route_colors=route_colors,
                route_linewidths=2,
                node_size=0,
                bgcolor='black',
                edge_color='dimgray',
                edge_linewidth=0.5,
                figsize=(15, 15),
                show=False,
                close=False
            )

            # Add marker for the fixed origin (incident/hotel)
            if 'hotel' in locals():
                 ax.scatter(hotel[1], hotel[0], c='white', s=200, label='Origin (Incident)', zorder=6, marker='*', edgecolor='black')

            # Add markers for each station
            plotted_station_labels = set()
            for marker_info in station_markers:
                coords = marker_info["coords"]
                color = marker_info["color"]
                label = marker_info["label"]
                if coords and len(coords) == 2:
                    display_label = label if label not in plotted_station_labels else None
                    ax.scatter(coords[1], coords[0], c=color, s=120, label=display_label, zorder=5, marker='X', edgecolor='white', linewidth=0.5)
                    plotted_station_labels.add(label)

            # Add a legend
            handles, labels = ax.get_legend_handles_labels()
            if handles:
                 fig.subplots_adjust(bottom=0.15)
                 ax.legend(handles=handles, labels=labels, loc='lower center',
                           bbox_to_anchor=(0.5, -0.1), fancybox=True, shadow=True,
                           ncol=min(len(handles), 6), title="Routes from Incident (V=Vehicle ID)")

            print("Displaying plot...")
            plt.show()
            print("Plot displayed.")

        except Exception as plot_error:
             print(f"\nAn error occurred during plotting: {plot_error}")
             print("Ensure 'G_undirected' has 'x' and 'y' node attributes.")


Preparing plot...
Filtering valid initial paths for plotting...

No valid initial paths found to plot.
